<a href="https://colab.research.google.com/github/scidatmath2020/AABD_SQL/blob/master/scripts/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pre Procesamiento de Datos**

In [ ]:
# Importando Librerias

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Importando Set de Entrenamiento
from google.colab import files
data_to_load = files.upload()

In [ ]:
import io
dataset_train  = pd.read_csv(io.BytesIO(data_to_load['Google_Stock_Price_Train.csv']))
training_set = dataset_train.iloc[:, 1:2].values

In [ ]:
# Escalado de Categorias

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)


In [ ]:
# Creando Estructura de Datos con 60 Pasos y 1 output

X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)


In [ ]:
# Remodelacion
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

 Armando la Red Neuronal Recurrente
**texto en negrita**

In [ ]:
# Importando Librerias
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


In [ ]:
# Iniciando RNR
regressor = Sequential()

In [ ]:
# Primera capa LSTM y Regularizacion de Dropout
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Segunda capa LSTM y Regularizacion de Dropout
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Tercera capa LSTM y Regularizacion de Dropout
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Cuarta capa LSTM y Regularizacion de Dropout
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))


In [ ]:
# Capa Output / Salida
regressor.add(Dense(units = 1))

In [ ]:
# Compilando RNR
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')


In [ ]:
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

**Haciendo Predicciones y Visualizando Datos**

In [ ]:
# Obteniendo los Precios de la Bolsa de Valores del Stock por Analizar
data_to_load = files.upload()


In [ ]:
dataset_test = pd.read_csv(io.BytesIO(data_to_load['Google_Stock_Price_Test.csv']))
real_stock_price = dataset_test.iloc[:, 1:2].values

In [ ]:
# Obteniendo el Precio Predicho
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
# Visualizando Resultados
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()
